# Predict Energy Behavior of Prosumers - Predictions

## 1. Import libraries

In [ ]:
# data processing
import numpy as np 
import pandas as pd
from geopy import distance
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import os

# modeling
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn import svm
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# evaluation
from sklearn.metrics import mean_squared_error, mean_absolute_error

# visualization
import plotly.express as px 

## 2. Import data

In [ ]:
data_dir = os.path.join(os.getcwd(), "data\\")

In [ ]:
prosumer = pd.read_csv(os.path.join(data_dir, 'prosumer.csv'))
weather_history = pd.read_csv(os.path.join(data_dir, 'weather_history.csv'))
weather_forecast = pd.read_csv(os.path.join(data_dir, 'weather_forecast.csv'))
gas_prices = pd.read_csv(os.path.join(data_dir, 'gas_prices.csv'))
electricity_prices = pd.read_csv(os.path.join(data_dir, 'electricity_prices.csv'))
client = pd.read_csv(os.path.join(data_dir, 'client.csv'))
counties = pd.read_json(os.path.join(data_dir, 'county_id_to_name_map.json'), typ="series")
weather_stations = pd.read_csv(os.path.join(data_dir, 'weather_station_to_county_mapping.csv'))

In [ ]:
# how big subset of the data to include in dataset
data_sample_size = 10000

## 3. Preprocess

### Add weather station data into weather data

Add information about the closest weather station into the weather data.

In [ ]:
weather_stations.head()

In [ ]:
print("Number of weather stations: ", len(weather_stations))

In [ ]:
print("Number stations with county info: ", len(weather_stations[~weather_stations["county"].isna()]))

**We need county info to connect weather data into prosumer data. Since, let's use only stations with county info available. 
For each weather data row, let's find the closest weather station with county info available.**

In [ ]:
def find_closest_weather_station(point: pd.Series, wss: pd.DataFrame):
    """
    Find closest weather station with county information to a point.
    """
    
    point_coordinates = [point.latitude, point.longitude]
    weather_station_coordinates = [[ws.latitude, ws.longitude] for ws in wss.itertuples()]
    
    # calculate distances to every weather station
    dists = [distance.distance(point_coordinates, station).km
             for station in weather_station_coordinates]
    # get closest station
    closest_dist = np.min(dists)
    closest_station = wss.iloc[np.argmin(dists), :]
    return closest_station, closest_dist
        

In [ ]:
def process_weather_data(wd: pd.DataFrame, wss: pd.DataFrame, feature_names: list):
    """
    Process weather data by
    - Adding weather station info to weather data by finding the closest weather station to a point
    """
    
    county_names = []
    county_ids = []
    for row in wd.itertuples():
        closest_station, dist = find_closest_weather_station(row, wss)
        if dist < 30:  # if station is less than 30 km away from point
            county_names.append(closest_station.county_name)
            county_ids.append(closest_station.county)
        else:
            county_names.append(None)
            county_ids.append(None)
    
    
    # add info to weather data
    wd_processed = wd.copy()
    wd_processed["county_name"] = county_names
    wd_processed["county"] = county_ids


    # take only data points from weather stations where county info is available
    wd_processed = wd_processed.dropna(subset=["county"], axis=0)

    # format timestamps
    wd_processed["forecast_datetime"] = wd_processed.forecast_datetime.apply(lambda x: pd.to_datetime(x).tz_localize(None).strftime("%Y-%m-%d %H:%M:%S"))

    # aggregate data per county, per timestamp
    wd_processed = wd_processed.groupby(["county", "forecast_datetime"])[feature_names].mean()

    return wd_processed

### Add capacity and consumption info for prosumers

In [ ]:
# capacity and consumption info is found in client.csv-file
client.head()

In [ ]:
def process_prosumer_data(prosumers: pd.DataFrame, clients: pd.DataFrame):
    """
    Process prosumer data by adding capacity and consumption info into it.
    """
    prosumers_proc = prosumers.copy()
    # add feature for date
    prosumers_proc["date"] = prosumers_proc.datetime.apply(lambda x: pd.to_datetime(x).strftime("%Y-%m-%d"))

    # calculate consumption and capacity per segment per date
    cons = client.groupby(["product_type", "county", "is_business", "date"])["eic_count"].sum()
    cap = client.groupby(["product_type", "county", "is_business", "date"])["installed_capacity"].sum()

    # add to prosumer data
    prosumers_proc = pd.merge(pd.merge(prosumers_proc, cons, on=["product_type", "county", "is_business", "date"]), cap, on=["product_type", "county", "is_business", "date"])

    return prosumers_proc

In [ ]:
def process_elec_price_data(data: pd.DataFrame):
    """
    Process electricity price data.
    """

    data_processed = data.rename(columns={"euros_per_mwh": "electricity_price"})
    return data_processed

In [ ]:
def process_gas_price_data(data: pd.DataFrame):
    """
    Process gas price data.
    """
    data_processed = data.copy()
    data_processed["avg_gas_price"] = data.apply(lambda row: np.mean([row.lowest_price_per_mwh, row.highest_price_per_mwh]), axis=1)
    return data_processed
    

## 4. Feature selection

### Make dataset

In [ ]:
def make_dataset(prosumer: pd.DataFrame, 
                 weather_forecast_data: pd.DataFrame, 
                 weather_stations: pd.DataFrame,
                 client: pd.DataFrame,
                 electricity_prices: pd.DataFrame,
                 gas_prices: pd.DataFrame,
                 weather_feature_names: list,
                 scaler
                 ):
    """
    Make dataset for predicting prosumer consumption and production.
    """

    # process data
    weather_data = process_weather_data(weather_forecast_data, weather_stations, weather_feature_names)
    prosumer_data = process_prosumer_data(prosumer, client)
    electricity_data = process_elec_price_data(electricity_prices)
    gas_data = process_gas_price_data(gas_prices)

    # combine together
    data = pd.merge(prosumer_data, 
                    weather_data, 
                    left_on=["county", "datetime"], 
                    right_index=True,
                    how="inner")

    # add price data
    data = pd.merge(
                pd.merge(data, 
                         electricity_data[["forecast_date", "electricity_price"]], 
                         left_on="datetime", 
                         right_on="forecast_date", 
                         how="left"), 
                gas_data[["forecast_date", "avg_gas_price"]], 
                left_on="date", 
                right_on="forecast_date", 
                how="left")

    # select features for dataset
    feats = ["is_consumption", "eic_count", "installed_capacity", "electricity_price", "avg_gas_price"] + weather_feature_names + ["target"]
    data = data[feats].reset_index(drop=True)

    if scaler is not None:
        target = data.target
        data = pd.DataFrame(scaler.fit_transform(data.drop("target", axis=1)))
        data["target"] = target

    return data    
    

In [ ]:
# select which weather parameters to use
weather_feat_names = ['direct_solar_radiation','surface_solar_radiation_downwards']

In [ ]:
# select scaler 
scaler = StandardScaler()

In [ ]:
weather_forecast

In [ ]:
# create dataset
dataset = make_dataset(prosumer.head(data_sample_size), 
                       weather_forecast.iloc[:data_sample_size, :], 
                       weather_stations, 
                       client, 
                       electricity_prices, 
                       gas_prices,
                       ["direct_solar_radiation", "surface_solar_radiation_downwards"],
                       scaler
                       )

In [ ]:
print("Size of the dataset: ", dataset.shape)

## 5. Modeling

Let's define three different models to be fit into training data. The fitted models are later used to predict prosumers' energy consumption and production.

Let's try to find the best model out of the three by applying the K-Fold method. In this method, each model is fit into subsets of the training data at the time, and their performance is evaluated
by predicting values for other subset of the data.


In [ ]:
def create_rf_model():
    """
    Create a Random Forest model
    """
    model = RandomForestRegressor()
    return model
    

In [ ]:
def create_svm_model():
    """
    Create a Support Vector Machine model
    """

    model = svm.SVR()
    return model

In [ ]:
def create_nn_model():
    """
    Create a neural network model
    """
    model = keras.Sequential()
    model.add(keras.layers.Dense(64))
    model.add(keras.layers.Dense(32))
    model.add(keras.layers.Dense(16))
    model.add(keras.layers.Dense(8))
    model.add(keras.layers.Dense(1))

    model.compile(optimizer="adam", loss="mae")

    return model

###  K-fold cross validation

In [ ]:
# split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(dataset.drop("target", axis=1), dataset.target, test_size=0.2)

In [ ]:
def cross_validate(data: pd.DataFrame, models, n_splits):
    """
    Cross validate different models to find out the best fitting model for the data.
    """

    kf = KFold(n_splits=10)
    # for storing results
    kfold_results = {}
    
    # define independent and dependent varibles
    X = dataset.iloc[:, :-1]
    y = dataset.target

    # cross validate
    run = 1
    for train_idx, test_idx in kf.split(X):
        scores = {}
        
        # define training and test sets
        X_train, X_test, y_train, y_test = X.iloc[train_idx, :], X.iloc[test_idx, :], y.iloc[train_idx], y.iloc[test_idx]
 
        # fit models and evaluate performaces
        for model_name in list(models.keys()):
            model = models.get(model_name)
            if model_name == "nn":    
                model.fit(X_train, y_train, verbose=0, epochs=20)
            else:
                model.fit(X_train, y_train)
            y_hat = model.predict(X_test)
            scores[model_name] = mean_absolute_error(y_test, y_hat)
            
        # store evaluation results
        kfold_results["Validation round {}".format(run)] = scores
        run += 1

    # find out the best model by taking mean score from all splits
    mean_scores = {model: sum(kfold_results[round_name][model] 
                              for round_name in kfold_results) / len(kfold_results) 
                   for model in kfold_results[list(kfold_results.keys())[0]]
                  }
    best_model_idx = np.argmin(list(mean_scores.values()))
    best_model_name = list(models.keys())[best_model_idx]
    best_model = models[best_model_name]
    best_model_score = mean_scores[best_model_name]

    return best_model, best_model_score
    

## 6. Fit and evaluate models

In [ ]:
best_model, score = cross_validate(pd.concat([X_train, y_train], axis=1), models={"rf": create_rf_model(), "smv": create_svm_model(), "nn": create_nn_model()}, n_splits=10)

In [ ]:
print("Best model: {}\nMean absolute error: {}".format(best_model, score))

### Predict with test data

In [ ]:
y_hat = best_model.predict(X_test)

### 7. Visualize predictions

In [ ]:
sample_size = 100  # how many data points to visualize

In [ ]:
results = pd.DataFrame(data=pd.concat([pd.Series(y_test.reset_index(drop=True)).iloc[:sample_size], pd.Series(y_hat).iloc[:sample_size]]), columns=["target"])

In [ ]:
results["type"] = pd.concat([pd.Series(np.repeat("Ground truth", sample_size)), pd.Series(np.repeat("Prediction", sample_size))]).values

In [ ]:
fig = px.line(results, 
              y="target", 
              color="type", 
              title="Predictions and true values for energy amount",
              labels={"target": "Energy amount", "index": "Timestep"},
              symbol="type",
              height=600
              )

In [ ]:
fig.update_traces(selector=({"name": "Prediction"}), opacity=.6)
fig.update_traces(selector=({"name": "Ground truth"}), opacity=.8)
fig.show()